In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings 
warnings.filterwarnings('ignore')

In [27]:
df = pd.read_csv("/Users/udit/git projects/ML---Implimentations/Adaboost Implementation/Dataset/cardekho_imputated.csv")

In [28]:
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [29]:
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [30]:
df.shape

(15411, 14)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15411 non-null  int64  
 1   car_name           15411 non-null  object 
 2   brand              15411 non-null  object 
 3   model              15411 non-null  object 
 4   vehicle_age        15411 non-null  int64  
 5   km_driven          15411 non-null  int64  
 6   seller_type        15411 non-null  object 
 7   fuel_type          15411 non-null  object 
 8   transmission_type  15411 non-null  object 
 9   mileage            15411 non-null  float64
 10  engine             15411 non-null  int64  
 11  max_power          15411 non-null  float64
 12  seats              15411 non-null  int64  
 13  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 1.6+ MB


In [32]:
# Remove Unnecessary Columns
df.drop(columns = ['car_name', 'brand', 'Unnamed: 0'], axis=1, inplace=True)

In [33]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [34]:
# Getting All Different Type Of Features 
num_features = [feature for feature in df.columns if df[feature].dtype != 'object']
print('no. of Numerical Features :', len(num_features))
cat_features = [feature for feature in df.columns if df[feature].dtype == 'object']
print('no. fo Categorical Features :', len(cat_features))
discrete_features = [feature for feature in num_features if len(df[feature].unique()) < 25]
print('no. of Discrete Feature :', len(discrete_features))
continuous_features = [feature for feature in num_features if feature not in discrete_features]
print('no. of Continuous Feature :', len(continuous_features))

no. of Numerical Features : 7
no. fo Categorical Features : 4
no. of Discrete Feature : 2
no. of Continuous Feature : 5


### Model Training

In [35]:
# Independent & Dependent features 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X = df.drop('selling_price', axis=1)
y = df['selling_price']

X['model'] = le.fit_transform(X['model'])

In [36]:
# create Transformer with 3 Types of Transformers
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.compose import ColumnTransformer 

num_features = X.select_dtypes(exclude='object').columns
ohe_features = ['seller_type','fuel_type','transmission_type']

numeric_transformer = StandardScaler() 
ohe_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("StandardScaler", numeric_transformer, num_features), 
        ("OneHotEncoder", ohe_transformer, ohe_features)
    ]
)

In [37]:
X = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.25, random_state=42)

X_train.shape, X_test.shape

((11558, 17), (3853, 17))

In [38]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor 
from sklearn.linear_model import LinearRegression, Ridge, Lasso 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor

In [39]:
# Create a function for Evaluate Model 
def evaluate_model(true, predicted): 
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true,predicted)
    return mae,mse,rmse,r2_square

In [40]:
# Model Training 
Models = { 
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbours Regressor" : KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "SVR" : SVR(),
    "AdaBoost Regressor" : AdaBoostRegressor(),
    "GradientBoost" : GradientBoostingRegressor()
}

In [41]:
for i in range(len(list(Models))): 
    model = list(Models.values())[i]
    model.fit(X_train,y_train)


    # Make Prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluate Train And Test Dataset
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(Models.keys())[i])

    print("Model Performance for Training set")
    print('- Mean Absolute Error : {:.4f}'.format(model_train_mae))
    print('- Mean Squared Error : {:.4f}'.format(model_train_mse))
    print('- Root Mean Squared Error : {:.4f}'.format(model_train_rmse))
    print('- R2 Score : {:.4f}'.format(model_train_r2))

    print('----------------------------------')


    print("Model Performance for Test set")
    print('- Mean Absolute Error : {:.4f}'.format(model_test_mae))
    print('- Mean Squared Error : {:.4f}'.format(model_test_mse))
    print('- Root Mean Squared Error : {:.4f}'.format(model_test_rmse))
    print('- R2 Score : {:.4f}'.format(model_test_r2))

    print("="*50)
    print("\n")



Linear Regression
Model Performance for Training set
- Mean Absolute Error : 266675.1076
- Mean Squared Error : 304874315292.8461
- Root Mean Squared Error : 552154.2495
- R2 Score : 0.6220
----------------------------------
Model Performance for Test set
- Mean Absolute Error : 284283.4460
- Mean Squared Error : 270286925822.7529
- Root Mean Squared Error : 519891.2635
- R2 Score : 0.6525


Lasso
Model Performance for Training set
- Mean Absolute Error : 266674.4040
- Mean Squared Error : 304874319723.5829
- Root Mean Squared Error : 552154.2536
- R2 Score : 0.6220
----------------------------------
Model Performance for Test set
- Mean Absolute Error : 284283.1476
- Mean Squared Error : 270286378251.9600
- Root Mean Squared Error : 519890.7368
- R2 Score : 0.6525


Ridge
Model Performance for Training set
- Mean Absolute Error : 266636.3543
- Mean Squared Error : 304875057099.0315
- Root Mean Squared Error : 552154.9213
- R2 Score : 0.6220
----------------------------------
Model Per

In [42]:
# Hyperparameter Tuning 
Rf_params = {
    "max_depth" : [5,8,15,None,10], 
    "max_features" : [5,7,"auto",8],
    "min_samples_split" : [2,8,15,20], 
    "n_estimators" : [100,200,500,1000]
}
GradientBoost_params = { 
    "loss" : ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'criterion' : ['friedman_mse', 'squared_error'],
    "min_samples_split" : [2,8,15,20], 
    "n_estimators" : [100,200,500,1000],
    "max_depth" : [5,8,15,None,10],
}

In [43]:
randomcv_models = [
    ("RF", RandomForestRegressor(), Rf_params),
    ("GradientBoost", GradientBoostingRegressor(), GradientBoost_params)
]

In [44]:
from sklearn.model_selection import RandomizedSearchCV 

model_params = {}
for name,model,params in randomcv_models: 
    random = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        cv = 3,
        n_iter=100,
        verbose=2,
        n_jobs=-1
    )
    random.fit(X_train,y_train)
    model_params[name] = random.best_params_

for model_name in model_params: 
    print(f'-------------------------Best Params from {model_name}---------------------------')
    print(model_params[model_name])

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END max_depth=15, max_features=auto, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_split=20, n_estimators=1000; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_split=20, n_estimators=1000; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_split=20, n_estimators=1000; total time=   0.0s
[CV] END max_depth=8, max_features=7, min_samples_split=15, n_estimators=100; total time=   0.5s
[CV] END max_depth=8, max_features=7, min_samples_split=15, n_estimators=100; total time=   0.5s
[CV] END max_depth=8, max_features=7, min_samples_split=15, n_estimators=100; total time=   0.6s
[CV] END max_depth=10, max_features=8

In [46]:
Models = { 
    "Random Forest Regressor" : RandomForestRegressor(n_estimators = 1000, min_samples_split = 2, max_features = 8, max_depth = 15),
    'GradientBoost' : GradientBoostingRegressor(n_estimators = 1000, min_samples_split = 2, max_depth = 5, loss = 'huber', criterion = 'friedman_mse')
}

for i in range(len(list(Models))): 
    model = list(Models.values())[i]
    model.fit(X_train,y_train)


    # Make Prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluate Train And Test Dataset
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(Models.keys())[i])

    print("Model Performance for Training set")
    print('- Mean Absolute Error : {:.4f}'.format(model_train_mae))
    print('- Mean Squared Error : {:.4f}'.format(model_train_mse))
    print('- Root Mean Squared Error : {:.4f}'.format(model_train_rmse))
    print('- R2 Score : {:.4f}'.format(model_train_r2))

    print('----------------------------------')


    print("Model Performance for Test set")
    print('- Mean Absolute Error : {:.4f}'.format(model_test_mae))
    print('- Mean Squared Error : {:.4f}'.format(model_test_mse))
    print('- Root Mean Squared Error : {:.4f}'.format(model_test_rmse))
    print('- R2 Score : {:.4f}'.format(model_test_r2))

    print("="*50)
    print("\n")


Random Forest Regressor
Model Performance for Training set
- Mean Absolute Error : 53926.0141
- Mean Squared Error : 20008145360.2546
- Root Mean Squared Error : 141450.1515
- R2 Score : 0.9752
----------------------------------
Model Performance for Test set
- Mean Absolute Error : 101039.5667
- Mean Squared Error : 52524684773.2229
- Root Mean Squared Error : 229182.6450
- R2 Score : 0.9325


GradientBoost
Model Performance for Training set
- Mean Absolute Error : 50400.1211
- Mean Squared Error : 6470326010.0202
- Root Mean Squared Error : 80438.3367
- R2 Score : 0.9920
----------------------------------
Model Performance for Test set
- Mean Absolute Error : 96845.5655
- Mean Squared Error : 52942082635.3510
- Root Mean Squared Error : 230091.4658
- R2 Score : 0.9319


